In [1]:
import cv2
from detection import yolo_v3_pipeline

path='data/'
video1='mcgill_drive.mp4'
video2='st-catherines_drive.mp4'

In [2]:
cap = cv2.VideoCapture(path+video1)

In [3]:
# while True:
#     ret, frame = cap.read()
#     if not ret: break
#     cv2.imshow('Frame', frame)
#     key = cv2.waitKey(0)
#     if key == 13 or key == 10:  
#         continue  # If 'Enter' key (key code 13 or 10) is pressed, show next frame
#     if key == ord('q') or key == 27: 
#         break   # If 'q' or 'Esc' key (key code 27) is pressed, exit the loop
# cap.release()
# cv2.destroyAllWindows()

In [4]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame through YOLOv3 pipeline
    processed_frame, labels, probabilities = yolo_v3_pipeline(frame)

    # Display the processed frame
    cv2.imshow("Processed Frame", processed_frame)

    # Exit on 'q' press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

P(car|Image) = 0.99754399061203
P(car|Image) = 0.9924140572547913
P(car|Image) = 0.9721500277519226
P(car|Image) = 0.869414210319519
P(car|Image) = 0.7616708874702454
P(person|Image) = 0.5766765475273132
P(car|Image) = 0.5606866478919983
P(car|Image) = 0.9975667595863342
P(car|Image) = 0.9782457947731018
P(car|Image) = 0.9580477476119995
P(car|Image) = 0.8857983946800232
P(car|Image) = 0.7175741791725159
P(car|Image) = 0.5678637027740479
P(car|Image) = 0.9975074529647827
P(car|Image) = 0.9763085246086121
P(car|Image) = 0.9237375855445862
P(car|Image) = 0.8893610835075378
P(truck|Image) = 0.7278700470924377
P(car|Image) = 0.634030818939209
P(car|Image) = 0.9983567595481873
P(car|Image) = 0.9877709746360779
P(car|Image) = 0.923258364200592
P(car|Image) = 0.8968266844749451
P(car|Image) = 0.683690071105957
P(truck|Image) = 0.6225572824478149
P(traffic light|Image) = 0.5423617959022522
P(car|Image) = 0.9980937838554382
P(car|Image) = 0.9924112558364868
P(car|Image) = 0.9225396513938904
P(c

KeyboardInterrupt: 